In [1]:
from utils import time_op
import impmeas as imp

solver = imp.formulas.GPMC()
imp.formulas.set_pmc_solver(solver)

In [2]:
# tests the model counting capabilities of SharpSAT-TD and Buddy

n,m,k = 8, 10, 7
cnf, formula = imp.random_k_cnf(n,m,k)
vars = [f"x{idx}" for idx in range(1, n+1)]

satcount_gpmc = time_op(solver.satcount)(cnf)
print(f"GPMC satcount = {satcount_gpmc}")

imp.formulas.set_buddy_context(vars)
f = time_op(imp.BuddyNode.parse)(formula)
satcount_bdd = time_op(f.expectation)() * 2**n
print(f"BDD satcount = {satcount_bdd}")

[00007.9231 ms / 0.0079 s / 0.0001 min] satcount
GPMC satcount = 252
[00076.3195 ms / 0.0763 s / 0.0013 min] parse
[00000.0069 ms / 0.0000 s / 0.0000 min] expectation
BDD satcount = 252.0


In [3]:
n,clauses_to_variables,k = 10, 4.5, 7
m = int(n*clauses_to_variables)
cnf, formula = imp.random_k_cnf(n,m,k)
vars = [f"x{idx+1}" for idx in range(n)]

In [4]:
import impmeas.bdds as bdd 
imp.formulas.set_buddy_context(vars)

x = "x1"
f = time_op(imp.BuddyNode.parse)(formula)
print(f"bdd size = {f.nodecount}")
blame = time_op(imp.blame)(f, x, rho=lambda x: 1/(x+1), cutoff=1e-4, modified=True, debug=True)
infl = time_op(imp.influence)(f, x)
print(f"influence {infl:.5f}, {blame:.5f} <= blame")

[00530.4010 ms / 0.5304 s / 0.0088 min] parse
bdd size = 118
=== COMPUTING BLAME for x1 in <impmeas.formulas.buddy.BuddyNode object at 0x7f7fc4e2c580> ===
k=0 d result=0.0938 max increase possible=0.4531 
k=1 d result=0.2061 max increase possible=0.1647 
k=2 d result=0.1426 max increase possible=0.0166 
k=3 d result=0.0166 max increase possible=0.0000 
stopped earlier because cannot improve above cutoff=0.0001.
current value: 0.4590, can only be increased by 0.0000.
=== DONE ===
[00030.9410 ms / 0.0309 s / 0.0005 min] blame
[00000.0408 ms / 0.0000 s / 0.0000 min] influence
influence 0.09375, 0.45898 <= blame


In [5]:
x = "x1"
f = time_op(imp.Formula.parse)(formula)
blame = time_op(imp.blame)(f, x, rho=lambda x: 1/(x+1), cutoff=1e-4, modified=True, debug=True)
infl = (time_op)(imp.influence)(f, x)
print(f"influence {infl:.5f}, {blame:.5f} <= blame")

[00559.7110 ms / 0.5597 s / 0.0093 min] parse
=== COMPUTING BLAME for x1 in Formula with size 1762 ===
k=0 size of cnf: 1270 d result=0.0938 max increase possible=0.4531 
k=1 size of cnf: 1270 d result=0.2061 max increase possible=0.1647 
k=2 size of cnf: 1270 d result=0.1426 max increase possible=0.0166 
k=3 size of cnf: 1270 d result=0.0166 max increase possible=0.0000 
stopped earlier because cannot improve above cutoff=0.0001.
current value: 0.4590, can only be increased by 0.0000.
=== DONE ===
[00285.8639 ms / 0.2859 s / 0.0048 min] blame
[00147.8815 ms / 0.1479 s / 0.0025 min] influence
influence 0.09375, 0.45898 <= blame


In [6]:

n,clauses_to_variables,k = 10, 4.5, 7
m = int(n*clauses_to_variables)
cnf, formula = imp.random_k_cnf(n,m,k)
vars = [f"x{idx+1}" for idx in range(n)]
x = "x1"

f = time_op(imp.Table.parse)(formula)
blame = time_op(imp.blame)(f, x, rho=lambda x: 1/(x+1), cutoff=1e-4, debug=False)
print(f"{blame:.5f} <= blame")
infl = time_op(imp.influence)(f, x)
print(f"influence = {infl:.5f}")
hammer = time_op(lambda f,x: imp.banzhaf(imp.hkr_cgm(f),x))(f, x)
print(f"hammer = {hammer:.5f}")
omega = time_op(imp.dominating_cgm)(f)
upsilon = time_op(imp.rectifying_cgm)(f)

print("="*100)
imp.formulas.set_buddy_context(vars)

f = time_op(imp.BuddyNode.parse)(formula)
print(f"bdd size = {f.nodecount}")
blame = time_op(imp.blame)(f, x, rho=lambda x: 1/(x+1), cutoff=1e-4, debug=True)
infl = time_op(imp.influence)(f, x)
print(f"influence {infl:.5f}, {blame:.5f} <= blame")


[00893.3971 ms / 0.8934 s / 0.0149 min] parse
[01373.5821 ms / 1.3736 s / 0.0229 min] blame
0.63525 <= blame
[00013.1588 ms / 0.0132 s / 0.0002 min] influence
influence = 0.34375
[06571.2042 ms / 6.5712 s / 0.1095 min] <lambda>
hammer = 0.07769
[00746.5584 ms / 0.7466 s / 0.0124 min] dominating_cgm
[00687.0615 ms / 0.6871 s / 0.0115 min] rectifying_cgm
[00619.1201 ms / 0.6191 s / 0.0103 min] parse
bdd size = 94
=== COMPUTING BLAME for x1 in <impmeas.formulas.buddy.BuddyNode object at 0x7f3c51f199c0> ===
k=0 d result=0.3438 max increase possible=0.3281 
k=1 d result=0.2227 max increase possible=0.0703 
k=2 d result=0.0645 max increase possible=0.0044 
k=3 d result=0.0044 max increase possible=0.0000 
stopped earlier because cannot improve above cutoff=0.0001.
current value: 0.6353, can only be increased by 0.0000.
=== DONE ===
[00163.1837 ms / 0.1632 s / 0.0027 min] blame
[00000.0658 ms / 0.0001 s / 0.0000 min] influence
influence 0.34375, 0.63525 <= blame
